# 导入库

In [161]:
import re, time
import requests
from bs4 import BeautifulSoup
from bs4.element import Tag
from aip import AipNlp
import csv
import pandas as pd

# 创建网页信息爬虫函数

In [3]:
"""
def getAuthor(data):
    rate = 0
    soup = BeautifulSoup(data, 'lxml')
    comments = soup.find_all('span', "comment-info")  # name rate
    for comment in comments:
        pattern = re.compile('span class="user-stars allstar(.*?) rating"')
        p = re.findall(pattern, str(comment))
        for star in p:
            rate = int(star)
            break
        soup = BeautifulSoup(str(comment), 'lxml')
        comments = soup.find_all('a')
        for item in comments:
            return item.string, rate
            break
        break
"""

In [45]:
def getContext(data):
    soup = BeautifulSoup(data, 'lxml')
    comments = soup.find_all("span", "short")
    for comment in comments:
        return comment.string
        break

# 参数信息填入

In [189]:
book_info={"射雕英雄传":"1789837","解忧杂货店":"25862578","追风筝的人":"1770782","平凡的世界（全三部）":"1200840","月亮和六便士":"1858513"}

In [46]:
""" 你的 APPID AK SK """
APP_ID = ' '
API_KEY = ''
SECRET_KEY = ' '

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

# 跑爬虫代码

In [220]:
comment_dict ={}
for book_name,book_id in book_info.items():
    index = 0
    count = 0
    NUM = 10
    alist=[]
    while count < NUM:
        r = requests.get('https://book.douban.com/subject/'+book_id+'/comments/hot?p='+str(index+1))
        index += 1
        soup = BeautifulSoup(r.text, 'lxml')
        comments = soup.find_all('li', 'comment-item')
        for comment in comments:
            context = getContext(str(comment))
            alist.append(context)
            count+=1
            if count >= NUM:
                break
        for x in alist:
            comment_dict.setdefault(book_name,[]).append(x)
        time.sleep(2)

In [221]:
comment_dict

{'射雕英雄传': ['我妈常说，宁和明白人打一架，不跟糊涂人说句话。这全书的智商都加在黄蓉身上了，导致江湖中人个个像SB一样误会来误会去，别人说啥信啥。金庸这上帝偏心都偏到后腰上了射雕的逻辑性还能看么。',
  '今日才知在这世上,欢喜快活原只一忽儿时光,愁苦烦恼才当真是一辈子的事。---黄蓉黄老师',
  '黃藥師好棒！擇偶標準！直男良心！以及周伯通真捉雞，想必王重陽在世時一定是個熊師兄，不然怎麼能帶出這種嗷嗷的熊孩子……',
  '电视的诸多版本，也比不上原著来的精彩。在文字里踏水行舟，一路所见岂非只是美景和快意恩仇。悟出些笑看人生的道理，其过程，像参透九阴真经或是武穆遗书之哪怕一行半句，也算小小精进一层。',
  '看到最后一页，成吉思汗被傻郭靖一句话逼死驾崩，不知道金庸老爷子写这个结局的时候，是怎样的心态。',
  '多好的蓉儿，等到《神雕》里嫁了郭靖、生了女儿也变大妈了。',
  '删掉蛤蛙大战真是因为怕被续吗？',
  '郭靖道：「蓉儿，你玩雕儿吧，我去杀散金兵。」',
  '天。用kindle看的，看的日子有点久还觉得自己现在越来越弱看书那么慢。过来标记才发现是4本，好的吧！爱死黄蓉了，数学好会做饭人聪明又专一，还特么好看！',
  '最好看的是黄蓉做饭，最感人的是黄老邪曾想过伴着《碧海潮生曲》，去找阿衡。'],
 '平凡的世界（全三部）': ['忍着终于听完，长吁一口气。“生活啊，你是多么的变幻莫测”一类无病呻吟听多了，李野墨语重心长的声音现在还赖在耳边不走。每当作者摆出振聋启聩的架势，等说出道理却不过是无谓与无根的感怀，我的耐心就被削去一层。写农村是书中最上乘的部分，有血有肉。但描述官场占据大段篇幅，人物如脸谱般单调羸弱，官话连篇难以下咽。我不赞成作者极力宣扬的价值观。人经受苦难不可怕，可怕的是对苦难的偏好。书中优秀的都市女性前赴后继拜倒在农村青年的高贵思想前，后者十分感动但是出于各种原因拒绝了她们。这是官宦千金爱上穷书生的翻版。出现一次是美好，可一个个都这样算怎么回事。作品的全知视角采取一种讲课口吻，文字似未精炼，叙述常重复，正如生怕学生忘了重点。假如你对自己的价值观延续千年没有十足把握，最好不要采取这么强势的叙述态度。',
  '理解不了这么高分都是怎么打的。匠气太重，自作聪明，充满革命现实主义那种莫名其妙的东西。语言平庸甚至有时做作，人物扁

# 情感分析 

In [222]:
"""
exam_data={'射雕英雄传': ['我妈常说，宁和明白人打一架，不跟糊涂人说句话。这全书的智商都加在黄蓉身上了，导致江湖中人个个像SB一样误会来误会去，别人说啥信啥。金庸这上帝偏心都偏到后腰上了射雕的逻辑性还能看么。',
  '今日才知在这世上,欢喜快活原只一忽儿时光,愁苦烦恼才当真是一辈子的事。---黄蓉黄老师',
  '黃藥師好棒！擇偶標準！直男良心！以及周伯通真捉雞，想必王重陽在世時一定是個熊師兄，不然怎麼能帶出這種嗷嗷的熊孩子……',
  '电视的诸多版本，也比不上原著来的精彩。在文字里踏水行舟，一路所见岂非只是美景和快意恩仇。悟出些笑看人生的道理，其过程，像参透九阴真经或是武穆遗书之哪怕一行半句，也算小小精进一层。',
  '看到最后一页，成吉思汗被傻郭靖一句话逼死驾崩，不知道金庸老爷子写这个结局的时候，是怎样的心态。']}
"""

"\nexam_data={'射雕英雄传': ['我妈常说，宁和明白人打一架，不跟糊涂人说句话。这全书的智商都加在黄蓉身上了，导致江湖中人个个像SB一样误会来误会去，别人说啥信啥。金庸这上帝偏心都偏到后腰上了射雕的逻辑性还能看么。',\n  '今日才知在这世上,欢喜快活原只一忽儿时光,愁苦烦恼才当真是一辈子的事。---黄蓉黄老师',\n  '黃藥師好棒！擇偶標準！直男良心！以及周伯通真捉雞，想必王重陽在世時一定是個熊師兄，不然怎麼能帶出這種嗷嗷的熊孩子……',\n  '电视的诸多版本，也比不上原著来的精彩。在文字里踏水行舟，一路所见岂非只是美景和快意恩仇。悟出些笑看人生的道理，其过程，像参透九阴真经或是武穆遗书之哪怕一行半句，也算小小精进一层。',\n  '看到最后一页，成吉思汗被傻郭靖一句话逼死驾崩，不知道金庸老爷子写这个结局的时候，是怎样的心态。']}\n"

In [223]:
all_comment=[]
for comment_dict_name in comment_dict.keys():
    for i in range(len(comment_dict[comment_dict_name])):
        sorted_comment=[]
        try:
            comment_content=client.sentimentClassify(comment_dict[comment_dict_name][i])
            if comment_content['items'][0]["sentiment"] == 0:
                emotion = "消极"
            elif comment_content['items'][0]["sentiment"] == 1 :
                emotion = "中性"
            elif comment_content['items'][0]["sentiment"] == 2 :
                emotion = "积极"
            else:
                print(comment_content)
            text = comment_content["text"]
            confidence = '{:.2f}%'.format(comment_content["items"][0]["confidence"]*100)
            # 积极和消极相减,保留小数点后两位,换成百分比
            difference_between = '{:.2f}%'.format((comment_content["items"][0]["positive_prob"] - comment_content["items"][0]["negative_prob"] )* 100)
            sorted_comment.append(comment_dict_name)
            sorted_comment.append(text) 
            sorted_comment.append(emotion)
            sorted_comment.append(confidence)
            sorted_comment.append(difference_between)
            all_comment.append(sorted_comment)
        except UnicodeEncodeError as err:
            pass
        #QPS 一秒限制5个
        time.sleep(1)

In [224]:
all_comment

[['射雕英雄传',
  '我妈常说，宁和明白人打一架，不跟糊涂人说句话。这全书的智商都加在黄蓉身上了，导致江湖中人个个像SB一样误会来误会去，别人说啥信啥。金庸这上帝偏心都偏到后腰上了射雕的逻辑性还能看么。',
  '消极',
  '88.76%',
  '-89.88%'],
 ['射雕英雄传',
  '今日才知在这世上,欢喜快活原只一忽儿时光,愁苦烦恼才当真是一辈子的事。---黄蓉黄老师',
  '积极',
  '51.35%',
  '56.21%'],
 ['射雕英雄传',
  '黃藥師好棒！擇偶標準！直男良心！以及周伯通真捉雞，想必王重陽在世時一定是個熊師兄，不然怎麼能帶出這種嗷嗷的熊孩子……',
  '积极',
  '64.23%',
  '67.80%'],
 ['射雕英雄传',
  '电视的诸多版本，也比不上原著来的精彩。在文字里踏水行舟，一路所见岂非只是美景和快意恩仇。悟出些笑看人生的道理，其过程，像参透九阴真经或是武穆遗书之哪怕一行半句，也算小小精进一层。',
  '消极',
  '70.07%',
  '-73.06%'],
 ['射雕英雄传',
  '看到最后一页，成吉思汗被傻郭靖一句话逼死驾崩，不知道金庸老爷子写这个结局的时候，是怎样的心态。',
  '消极',
  '87.65%',
  '-88.89%'],
 ['射雕英雄传', '多好的蓉儿，等到《神雕》里嫁了郭靖、生了女儿也变大妈了。', '中性', '48.73%', '-5.13%'],
 ['射雕英雄传', '删掉蛤蛙大战真是因为怕被续吗？', '消极', '48.22%', '-53.40%'],
 ['射雕英雄传', '郭靖道：「蓉儿，你玩雕儿吧，我去杀散金兵。」', '积极', '21.26%', '29.14%'],
 ['射雕英雄传',
  '天。用kindle看的，看的日子有点久还觉得自己现在越来越弱看书那么慢。过来标记才发现是4本，好的吧！爱死黄蓉了，数学好会做饭人聪明又专一，还特么好看！',
  '积极',
  '82.16%',
  '83.94%'],
 ['射雕英雄传', '最好看的是黄蓉做饭，最感人的是黄老邪曾想过伴着《碧海潮生曲》，去找阿衡。', '积极', '89.14%', '90.22%'],
 ['解忧杂货店', '真的要到很久以后，才会明

# 存档

In [227]:
with open ("douban_commment_data.tsv",'w', encoding="utf8")as gg:
    for x in  range(len(all_comment)):
        new=all_comment[x][0]+"\t"+all_comment[x][1]+"\t"+all_comment[x][2]+"\t"+all_comment[x][3]+"\t"+all_comment[x][4]+"\n"
        gg.write(new)

# 查看评价

In [228]:
df = pd.read_csv("douban_commment_data.tsv", delimiter='\t',names=['书名','评论','情感倾向','可信度','相差度/(积极概率-消极概率/)'])
df

,书名,评论,情感倾向,可信度,相差度/(积极概率-消极概率/)
0,射雕英雄传,我妈常说，宁和明白人打一架，不跟糊涂人说句话。这全书的智商都加在黄蓉身上了，导致江湖中人个个...,消极,88.76%,-89.88%
1,射雕英雄传,"今日才知在这世上,欢喜快活原只一忽儿时光,愁苦烦恼才当真是一辈子的事。---黄蓉黄老师",积极,51.35%,56.21%
2,射雕英雄传,黃藥師好棒！擇偶標準！直男良心！以及周伯通真捉雞，想必王重陽在世時一定是個熊師兄，不然怎麼能...,积极,64.23%,67.80%
3,射雕英雄传,电视的诸多版本，也比不上原著来的精彩。在文字里踏水行舟，一路所见岂非只是美景和快意恩仇。悟出...,消极,70.07%,-73.06%
4,射雕英雄传,看到最后一页，成吉思汗被傻郭靖一句话逼死驾崩，不知道金庸老爷子写这个结局的时候，是怎样的心态。,消极,87.65%,-88.89%
5,射雕英雄传,多好的蓉儿，等到《神雕》里嫁了郭靖、生了女儿也变大妈了。,中性,48.73%,-5.13%
6,射雕英雄传,删掉蛤蛙大战真是因为怕被续吗？,消极,48.22%,-53.40%
7,射雕英雄传,郭靖道：「蓉儿，你玩雕儿吧，我去杀散金兵。」,积极,21.26%,29.14%
8,射雕英雄传,天。用kindle看的，看的日子有点久还觉得自己现在越来越弱看书那么慢。过来标记才发现是4本...,积极,82.16%,83.94%
9,射雕英雄传,最好看的是黄蓉做饭，最感人的是黄老邪曾想过伴着《碧海潮生曲》，去找阿衡。,积极,89.14%,90.22%
